In [ ]:
import sys
import pandas as pd
import numpy as np
import datetime
import math
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN, GRU,TimeDistributed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
import tensorflow as tf
import collections
import os, gc
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

def model_creator(type_of_RNN="LSTM", dense_size=14, input_shape=(1,16), dropout=True, drop_rate=0.5, loss='categorical_crossentropy', optimizer="Adam", lr=0.01, number_of_classes=2):
    model = Sequential()
    if type_of_RNN=="LSTM":
        model.add(LSTM(dense_size, input_shape=input_shape))
    if type_of_RNN=="RNN":
        model.add(SimpleRNN(dense_size, input_shape=input_shape))
    if type_of_RNN=="GRU":
        model.add(GRU(dense_size, input_shape=input_shape))
    if dropout:
        model.add(Dropout(drop_rate))
    model.add(Dense(number_of_classes, activation='softmax'))
    
    if optimizer=="Adam":
        optimizer=optimizers.Adam(lr)
    if optimizer=="RMSprop":
        optimizer=optimizers.RMSprop(lr)
    if optimizer=="SGD":
        optimizer=optimizers.SGD(lr)
    if optimizer=="Adagrad":
        optimizer=optimizers.Adagrad(lr)
    if optimizer=="Adadelta":
        optimizer=optimizers.Adadelta(lr)
    model.compile(loss=loss, optimizer=optimizer, 
                  metrics=[])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

my_classifier = KerasClassifier(model_creator, batch_size=1024)
validator = GridSearchCV(my_classifier,
                         param_grid={
                                    'type_of_RNN': ["LSTM","RNN","GRU"],
                                     'optimizer': ["Adam","RMSprop"],
                                     'epochs': [10],
                                     'validation_split':[0.1],
                                     'batch_size':[1024],
                                     'number_of_classes':[2],
                                     'dropout': [False, True],
                                     'dense_size': [14],
                                     'lr': [0.001]
        
                        },
                         cv=1,
                         verbose=27,
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(X_train[:10000], to_categorical(y_train[:10000,1]))

In [ ]:
name="COD_SEER_model"
model_checkpoint = ModelCheckpoint("./COD_SEER_model/weights_%s-{epoch:02d}.h5" % (name), monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
csv_logger = CSVLogger('training_{name}.log')
# learning rate schedule
def smoothed_categorical_crossentropy(y_true, y_pred):
    return keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=False, label_smoothing=0.2) 
def step_decay(epoch, lr):
    initial_lrate = 0.001
    drop = 0.8
    epochs_drop = 10.0
    lrate = lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)
model_n = Sequential()
model_n.add(SimpleRNN(14, input_shape=(1,16)))
model_n.add(Dense(1, activation='linear'))
model_n.compile(loss="mse", optimizer=optimizers.Adam(0.001), metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.SpecificityAtSensitivity(1.0) ])

histo_=model_n.fit(X_train_selected,y_train[:,1], epochs=10, batch_size=1024, validation_split=0.01,callbacks=[lrate,csv_logger,model_checkpoint])

In [ ]:
y_pred_rnn=model_all.predict(X_train_selected, batch_size=1024)

In [ ]:
def GetRNNSurvivalFunction(x):
    kmf_ = KaplanMeierFitter() 
    kmf_.fit(X_train_selected[:,:,-1], y_pred_rnn[:,1]+x, label="Predicted Kaplan Meier Estimate - RNN - ")

    kmf = KaplanMeierFitter() 
    kmf.fit(X_train_selected[:,:,-1], Y_train_selected[:,1], label="Predicted Kaplan Meier Estimate - True - ")
    
    RNN_KM_estimate = kmf_.confidence_interval_.to_dict()
    Normal_KM_estimate = kmf.confidence_interval_.to_dict()
    
    return np.mean(np.subtract(list(RNN_KM_estimate[key_RNN[0]].values()),list(Normal_KM_estimate[key_KM[0]].values())))**2

In [ ]:
from scipy import optimize
res=optimize.minimize_scalar(GetF,method='brent') #Get local minimum for the best calibrated model

In [ ]:
from lifelines import KaplanMeierFitter
kmf_ = KaplanMeierFitter() 
kmf_.fit(X_train_selected[:,:,-1], y_pred_rnn[:,1]+res.x, label="Predicted Kaplan Meier Estimate - RNN - ")
a2=kmf_.plot()
kmf = KaplanMeierFitter() 
kmf.fit(X_train_selected[:,:,-1], Y_train_selected[:,1], label="Predicted Kaplan Meier Estimate - True - ")
a2=kmf.plot()